# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [8]:
# import libraries
import datetime
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#! pip install pyspark
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import udf, col, concat, count, lit, avg, lag, first, last, when, desc, asc, countDistinct, when, isnan
from pyspark.sql.functions import min as Fmin, max as Fmax, sum as Fsum, round as Fround

from pyspark.sql.types import IntegerType, FloatType, DateType, TimestampType, StringType

from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler

from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier, LinearSVC, LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

from pyspark.ml.regression import LinearRegression


%matplotlib inline

In [27]:
# create a Spark session
spark = SparkSession \
        .builder \
        .appName("Sparkify") \
        .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [30]:
path = "mini_sparkify_event_data.json"
sparkify_events = spark.read.json(path)

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

In [31]:
sparkify_events.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [32]:
print("The number of rows is", sparkify_events.count())
print("The number of columns is", len(sparkify_events.columns))

The number of rows is 286500
The number of columns is 18


In [33]:
# Check the unique values of some categories to have an idea of the data
for col in ['auth', 'gender', 'location', 'level', 'method', 'page', 'status', 'registration']:
    print(sparkify_events.groupby(col).count().show(30))

+----------+------+
|      auth| count|
+----------+------+
|Logged Out|  8249|
| Cancelled|    52|
|     Guest|    97|
| Logged In|278102|
+----------+------+

None
+------+------+
|gender| count|
+------+------+
|     F|154578|
|  null|  8346|
|     M|123576|
+------+------+

None
+--------------------+-----+
|            location|count|
+--------------------+-----+
|     Gainesville, FL| 1229|
|Atlantic City-Ham...| 2176|
|Deltona-Daytona B...|   73|
|San Diego-Carlsba...|  754|
|Cleveland-Elyria, OH| 1392|
|Kingsport-Bristol...| 1863|
|New Haven-Milford...| 4007|
|Birmingham-Hoover...|   75|
|  Corpus Christi, TX|   11|
|         Dubuque, IA|  651|
|Las Vegas-Henders...| 2042|
|Indianapolis-Carm...|  970|
|Seattle-Tacoma-Be...|  246|
|          Albany, OR|   23|
|   Winston-Salem, NC|  819|
|     Bakersfield, CA| 1775|
|Los Angeles-Long ...|30131|
|Minneapolis-St. P...| 2134|
|San Francisco-Oak...| 2647|
|Phoenix-Mesa-Scot...| 4846|
|Allentown-Bethleh...|  815|
|Miami-Fort Lauder..

In [34]:
# Print first 2 lines of 'sparkify_events'
sparkify_events.show(n=2, truncate=False, vertical=True)

-RECORD 0------------------------------------------------------------------------------------------------------------------------
 artist        | Martha Tilston                                                                                                  
 auth          | Logged In                                                                                                       
 firstName     | Colin                                                                                                           
 gender        | M                                                                                                               
 itemInSession | 50                                                                                                              
 lastName      | Freeman                                                                                                         
 length        | 277.89016                                                                

### Clean Data

#### Missing values

In [35]:
# Check columns with NaN values
for value in sparkify_events.columns:
    print ("In column (",value,") the number of missing values is", sparkify_events.where(sparkify_events[value].isNull()).count())

In column ( artist ) the number of missing values is 58392
In column ( auth ) the number of missing values is 0
In column ( firstName ) the number of missing values is 8346
In column ( gender ) the number of missing values is 8346
In column ( itemInSession ) the number of missing values is 0
In column ( lastName ) the number of missing values is 8346
In column ( length ) the number of missing values is 58392
In column ( level ) the number of missing values is 0
In column ( location ) the number of missing values is 8346
In column ( method ) the number of missing values is 0
In column ( page ) the number of missing values is 0
In column ( registration ) the number of missing values is 8346
In column ( sessionId ) the number of missing values is 0
In column ( song ) the number of missing values is 58392
In column ( status ) the number of missing values is 0
In column ( ts ) the number of missing values is 0
In column ( userAgent ) the number of missing values is 8346
In column ( userId )

#### Empty values

In [36]:
# Check columns with empty values
for value in sparkify_events.columns:
    print ("In column (",value,") the number of empty values is", sparkify_events.filter(sparkify_events[value] == '').count())

In column ( artist ) the number of empty values is 0
In column ( auth ) the number of empty values is 0
In column ( firstName ) the number of empty values is 0
In column ( gender ) the number of empty values is 0
In column ( itemInSession ) the number of empty values is 0
In column ( lastName ) the number of empty values is 0
In column ( length ) the number of empty values is 0
In column ( level ) the number of empty values is 0
In column ( location ) the number of empty values is 0
In column ( method ) the number of empty values is 0
In column ( page ) the number of empty values is 0
In column ( registration ) the number of empty values is 0
In column ( sessionId ) the number of empty values is 0
In column ( song ) the number of empty values is 0
In column ( status ) the number of empty values is 0
In column ( ts ) the number of empty values is 0
In column ( userAgent ) the number of empty values is 0
In column ( userId ) the number of empty values is 8346


In [37]:
# Delete columns with empty userid
sparkify_events = sparkify_events.filter(sparkify_events.userId != "")

In [38]:
# Check columns with empty values
for value in sparkify_events.columns:
    print ("In column (",value,") the number of empty values is", sparkify_events.filter(sparkify_events[value] == '').count())

In column ( artist ) the number of empty values is 0
In column ( auth ) the number of empty values is 0
In column ( firstName ) the number of empty values is 0
In column ( gender ) the number of empty values is 0
In column ( itemInSession ) the number of empty values is 0
In column ( lastName ) the number of empty values is 0
In column ( length ) the number of empty values is 0
In column ( level ) the number of empty values is 0
In column ( location ) the number of empty values is 0
In column ( method ) the number of empty values is 0
In column ( page ) the number of empty values is 0
In column ( registration ) the number of empty values is 0
In column ( sessionId ) the number of empty values is 0
In column ( song ) the number of empty values is 0
In column ( status ) the number of empty values is 0
In column ( ts ) the number of empty values is 0
In column ( userAgent ) the number of empty values is 0
In column ( userId ) the number of empty values is 0


We got rid of the userId rows that were empty. Now let's take a look at the NaN values

In [40]:
# Check columns with NaN values
for value in sparkify_events.columns:
    print ("In column (",value,") the number of missing values is", sparkify_events.where(sparkify_events[value].isNull()).count())

In column ( artist ) the number of missing values is 50046
In column ( auth ) the number of missing values is 0
In column ( firstName ) the number of missing values is 0
In column ( gender ) the number of missing values is 0
In column ( itemInSession ) the number of missing values is 0
In column ( lastName ) the number of missing values is 0
In column ( length ) the number of missing values is 50046
In column ( level ) the number of missing values is 0
In column ( location ) the number of missing values is 0
In column ( method ) the number of missing values is 0
In column ( page ) the number of missing values is 0
In column ( registration ) the number of missing values is 0
In column ( sessionId ) the number of missing values is 0
In column ( song ) the number of missing values is 50046
In column ( status ) the number of missing values is 0
In column ( ts ) the number of missing values is 0
In column ( userAgent ) the number of missing values is 0
In column ( userId ) the number of mis

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

#### Cancellation Confirmation

In [42]:
# Flag definition for churn (Cancellation Confirmation)
churn_flag = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
sparkify_events = sparkify_events.withColumn("churn", churn_flag("page"))

# Apply churn flag to dataset
window_val = Window.partitionBy("userId")
sparkify_events = sparkify_events.withColumn("churn", Fsum("churn").over(window_val))

In [46]:
# Churned users
sparkify_events["userId", "churn"].dropDuplicates().groupby("churn").count().show()

+-----+-----+
|churn|count|
+-----+-----+
|    0|  173|
|    1|   52|
+-----+-----+



#### Downgrade Events

In [44]:
# Flag definition for churn (Downgrade)
churn_downgrade = udf(lambda x: 1 if x == "Downgrade" else 0, IntegerType())
sparkify_events = sparkify_events.withColumn("downgrade", churn_downgrade("page"))

# Apply churn flag to dataset
windowval_val = Window.partitionBy("userId")
sparkify_events = sparkify_events.withColumn("downgrade", Fsum("downgrade").over(windowval_val))

In [ ]:
# Downgraded users
sparkify_events["userId", "downgrade"].dropDuplicates().groupby("downgrade").count().show()

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.